In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import transforms, models
from data_loader import FERDataset
import transform
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import time
import copy
import numpy as np
import cv2 as cv

plt.rcParams['figure.figsize'] = (20, 20)
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

print('Torch version: {}'.format(torch.__version__))
print('Torchvision version: {}'.format(torchvision.__version__))
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('GPU: {}'.format(device))

In [ ]:
data_transforms = transforms.Compose([
    transform.Normalize(),
    transform.ToTensor()
])

train = FERDataset('fer2013.csv', 'Training', transform=data_transforms)
val = FERDataset('fer2013.csv', 'PublicTest', transform=data_transforms)
# test = FERDataset('fer2013.csv', 'PrivateTest', transform=data_transforms)
print(len(train), len(val))

In [ ]:
bs = 32
train_loader = DataLoader(train, batch_size=bs, shuffle=True)
val_loader = DataLoader(val, batch_size=bs, shuffle=False)

In [ ]:
classes = ('Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral')
num_samples = 4
for idx, sample in enumerate(train_loader):
    if idx == num_samples:
        break
    images, labels = sample['image'], sample['label']
    image = torch.squeeze(images[0]) * 255
    plt.subplot(1, num_samples, idx+1)
    plt.imshow(image, cmap='gray')
    plt.title(classes[int(labels[0].numpy())])
    plt.axis('off')
plt.show()

In [ ]:
class EmoNet(nn.Module):

    def __init__(self):
        super(EmoNet, self).__init__()
        self.reset()
        self.features = nn.Sequential(
            nn.Conv2d(1, 16, 3, padding=1),
            nn.BatchNorm2d(16),
            nn.MaxPool2d(2, 2),
            nn.ReLU(),
            nn.Dropout(0.1),

            nn.Conv2d(16, 32, 3, padding=1),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(2, 2),
            nn.ReLU(),
            nn.Dropout(0.2),
            
            nn.Conv2d(32, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(2, 2),
            nn.ReLU(),
            nn.Dropout(0.3),
            
            nn.Conv2d(64, 128, 3, padding=1),
            nn.BatchNorm2d(128),
            nn.MaxPool2d(2, 2),
            nn.ReLU(),
            nn.Dropout(0.4)
        )
        self.classifier = nn.Sequential(
            nn.Linear(128*3*3, 200),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(200, 7)
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(-1, self.num_flat_features(x))
        x = self.classifier(x)
        return x
    
    def num_flat_features(self, x):
        n_feats = 1
        for s in x.size()[1:]:
            n_feats *= s

        return n_feats
    
    def reset(self):
        self.train_loss_history = []
        self.train_acc_history = []
        self.val_loss_history = []
        self.val_acc_history = []

    def print_params(self):
        total_params = sum(param.numel() for param in self.parameters())
        trainable = sum(param.numel() for param in self.parameters() if param.requires_grad == True)
        print('Total params: {}\nTrainable params: {}'.format(total_params, trainable))

In [ ]:
model = EmoNet()
print(model)
model.print_params()
model = model.to(device)

In [ ]:
def train_net(model, train_loader, val_loader, use_cuda, print_every, n_epochs):
    print('\nStarted training...\n')
    best_val_acc = 0.50
    best_model_wts = copy.deepcopy(model.state_dict())
    for epoch in range(n_epochs):
        model.train
        running_train_loss = 0.0
        for batch, sample in enumerate(train_loader):
            images, labels = sample['image'], sample['label']
            if use_cuda:
                images = images.to(device, dtype=torch.float)
                labels = labels.to(device)

            output = model(images)
            train_loss = criterion(output, labels.squeeze(1))
            running_train_loss += train_loss.item()
            
            optimizer.zero_grad()
            train_loss.backward()
            optimizer.step()

            if (batch+1) % print_every == 0:
                print('[Iteration {}/{}] Training loss: {:.3f}'.format(batch+1, len(train_loader), train_loss.item()))
        model.train_loss_history.append(running_train_loss/len(train_loader))
        with torch.no_grad():
            model.eval
            running_val_loss = 0.0
            val_acc = 0.0
            for batch, sample in enumerate(val_loader):
                image_val, label_val = sample['image'], sample['label']
                if use_cuda:
                    image_val = image_val.to(device, dtype=torch.float)
                    label_val = label_val.to(device)
                    
                o_val = model(image_val)
                val_loss = criterion(o_val, label_val.squeeze(1))
                running_val_loss += val_loss.item()
                
                total = label_val.size(0)
                _, prediction = torch.max(o_val.data, 1)
                correct = (prediction == label_val.squeeze(1)).sum().item()
                val_acc += (correct/total)
                
            model.val_loss_history.append(running_val_loss/len(val_loader))
            model.val_acc_history.append(val_acc/len(val_loader))
        print('[Epoch {}/{}] Training loss: {:.3f}, Validation loss: {:.3f}, Validation acc: {:.3f}\n'.format(epoch+1, n_epochs, model.train_loss_history[-1], model.val_loss_history[-1], model.val_acc_history[-1]))
        
        if model.val_acc_history[-1] >= best_val_acc:
            best_val_acc = model.val_acc_history[-1]
            best_model_wts = copy.deepcopy(model.state_dict())
            print('Achieved better validation acc. Saving state...')
    print('\nFinished training...\n')
    model.load_state_dict(best_model_wts)
    return model

In [ ]:
since = time.time()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-8, weight_decay=1e-6)
best_model = train_net(model, train_loader, val_loader, True, 200, 10)
torch.save(best_model, 'Saved_Models/model1.pt')
time_elapsed = time.time() - since
print('Total training time: {}m {}s'.format(time_elapsed // 60, time_elapsed % 60))

In [ ]:
plt.plot(model.train_loss_history)
plt.plot(model.val_loss_history)
plt.legend(['train', 'val'])
plt.title('Training vs. Validation loss')
plt.show()

In [ ]:
face_cascade = cv.CascadeClassifier('/home/hashir/anaconda3/lib/python3.7/site-packages/cv2/data/haarcascade_frontalface_default.xml')
saved_model = torch.load('Saved_Models/model1.pt')
saved_model.eval

In [ ]:
classes = ('Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral')
cap = cv.VideoCapture(0)
time.sleep(2.0)

while cap.isOpened():
    ret, frame = cap.read()
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    for (x,y,w,h) in faces:
        cv.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
        face_gray = gray[y:y+h, x:x+w]
        sample = cv.resize(face_gray, (48, 48))
        sample = sample.astype('float32')/255.
        sample = np.asarray(sample).reshape(1,48,48)
        sample = torch.from_numpy(sample).unsqueeze(0).to(device)
        output = saved_model(sample)
        _, prediction = torch.max(output.data, 1)
        label = classes[prediction]
        cv.putText(frame, str(label), (x-10, y-20), cv.FONT_HERSHEY_SIMPLEX, 1.2, (0,0,255), 1, cv.LINE_AA)
    cv.imshow("Frame", frame)
    key = cv.waitKey(1) & 0xFF
    
    if key == ord('q'):
        break
        
cap.release()
cv.destroyAllWindows()
cv.waitKey(1)